In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/liar-fake-news-dataset/test.tsv
/kaggle/input/liar-fake-news-dataset/README
/kaggle/input/liar-fake-news-dataset/train.tsv
/kaggle/input/liar-fake-news-dataset/valid.tsv
/kaggle/input/translated-data-nepali/translated_data_nepali.tsv


In [3]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt 


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
df = pd.read_csv('/kaggle/input/liar-fake-news-dataset/train.tsv', sep='\t')

In [5]:
df.head()

,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0.1,0.2,0.3,a mailer
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [6]:
df.shape

(10239, 14)

In [7]:
for col in df.columns:
    print(col)

2635.json
false
Says the Annies List political group supports third-trimester abortions on demand.
abortion
dwayne-bohac
State representative
Texas
republican
0
1
0.1
0.2
0.3
a mailer


In [8]:
df.columns=['id', 'label', 'statement', 'subjects','speaker', 'speaker_job_title', 'state_info', 'party_affilation',
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement) 
           ]
df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [9]:
df.value_counts('label')

label
half-true      2114
false          1994
mostly-true    1962
true           1676
barely-true    1654
pants-fire      839
dtype: int64

In [10]:
!pip install -U deep-translator

In [11]:
from deep_translator import GoogleTranslator

In [12]:
translated = GoogleTranslator(source='auto', target='ne').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig
translated

'यसलाई जारी राख्नुहोस्, तपाईं अद्भुत हुनुहुन्छ'

In [13]:
random =  df.sample(10)
random['statement'].iloc[0]


'Says an appointed board created under Obamacare will have the ability to come between you and your doctor in determining the best treatment options for you!'

In [14]:
translated = GoogleTranslator(source='auto', target='ne').translate(random['statement'].iloc[0])  # output -> Weiter so, du bist großartig
translated

'ओबामाकेयर अन्तर्गत बनाइएको नियुक्त बोर्डले तपाईंको लागि उत्तम उपचार विकल्पहरू निर्धारण गर्न तपाईं र तपाईंको डाक्टरको बीचमा आउन सक्ने क्षमता हुनेछ!'

In [15]:
random['translated_value'] = random['statement'].apply(lambda x: GoogleTranslator(source='auto', target='ne').translate(x))

In [16]:
random['translated_value']

7753    ओबामाकेयर अन्तर्गत बनाइएको नियुक्त बोर्डले तपा...
217     रिक स्कटले ड्रिलिंगको लागि जग्गा भाडामा लिएको ...
4102    आज जन्मिएको कालो पुरुष बच्चा, यदि हामीले प्रणा...
1111    आपत्‌कालीन कोठामा हुने क्षतिपूर्ति नभएको हेरचा...
687     अमेरिकी जनताले चासो राखेको नम्बर १ मुद्दा भनेक...
6327    ओम्निबस खर्च बिलमा 9,427 पोर्क ब्यारेल वस्तुहर...
1955    Freshman Rep. Bob Gibbs ले आफ्नो एक वर्षको वार...
4749    सिनेटर वेब राष्ट्रपति ओबामा र सिनेटर ह्यारी री...
9686    त्यसपछि-सेन। बराक ओबामाले ऋणको सीमा बढाउन अस्व...
9086    40 वर्षमा, 23,000 रसायनहरू बजारमा आए र केवल चा...
Name: translated_value, dtype: object

In [17]:
random['retranslated_value'] = random['translated_value'].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))


In [18]:
random['retranslated_value'].iloc[0]

'An appointed board created under Obamacare will have the ability to come between you and your doctor to determine the best treatment options for you!'

In [19]:
random['statement'].iloc[2]

'A black male baby born today, if we do not change the system, stands a one-in-three chance (of) ending up in jail.'

In [20]:
index=2
print(random['statement'].iloc[index], "\n", random['translated_value'].iloc[index], "\n",
random["retranslated_value"].iloc[index])

A black male baby born today, if we do not change the system, stands a one-in-three chance (of) ending up in jail. 
 आज जन्मिएको कालो पुरुष बच्चा, यदि हामीले प्रणाली परिवर्तन गरेनौं भने, जेलमा समाप्त हुने तीनमा एक (तीन) मौका खडा हुन्छ। 
 A black male child born today, if we don't change the system, stands a one (three) chance of ending up in prison.


In [21]:
index=3
print(random['statement'].iloc[index], "\n", random['translated_value'].iloc[index], "\n",
random["retranslated_value"].iloc[index])

Its costing every American who is insured $1,100 to pick up the cost of uncompensated care that goes on at the emergency room. 
 आपत्‌कालीन कोठामा हुने क्षतिपूर्ति नभएको हेरचाहको लागत उठाउन $1,100 बीमा गरिएको प्रत्येक अमेरिकीलाई यसको लागत छ। 
 It costs each insured American $1,100 to cover the cost of uncompensated emergency room care.


In [22]:
!pip install inflect


In [23]:
new_df = df.copy()
new_df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [24]:
from deep_translator import GoogleTranslator
import inflect

def translate_to_nepali(text, target_lang):
    # Convert numbers to text using the inflect library
    p = inflect.engine()
    words = text.split()
    for i, word in enumerate(words):
        if word.isnumeric():
            words[i] = p.number_to_words(word)
    text = ' '.join(words)
    
    # Translate the text using GoogleTranslator
    translated_text = GoogleTranslator(source='auto', target=target_lang).translate(text)
    
    return translated_text



In [ ]:
new_df['nepali_translation'] = new_df['statement'].apply(lambda x: translate_to_nepali(x,"ne"))


In [ ]:
new_df.head()

In [ ]:
new_df.to_csv('/kaggle/working/translated_data_nepali.tsv', sep='\t', index=False)


In [26]:
df1 = pd.read_csv('/kaggle/input/translated-data-nepali/translated_data_nepali.tsv', sep='\t')

In [27]:
df1.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context,nepali_translation
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,कोइलाको गिरावट कहिले सुरु भयो? यो सुरु भयो जब ...
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,"हिलारी क्लिन्टनले जोन म्याककेनसँग ""जर्ज बुशलाई..."
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,स्वास्थ्य हेरचाह सुधार कानूनले नि: शुल्क लिंग ...
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,मेरो कार्यकालको अन्त्यमा आर्थिक परिवर्तन सुरु ...
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,शिकागो बियर्सले विगत दुई दशकहरूमा बर्खास्त गरि...


In [28]:
df1.shape

(10239, 15)

In [29]:
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 4.1 MB/s eta 0:00:00


In [38]:
new_df1 = df1.copy()
new_df1['nepali_translation'].iloc(0)

In [43]:
new_df1['nepali_translation'][10]

'2000 यता, लगभग 12 मिलियन अमेरिकीहरू मध्यम वर्ग र गरिबीमा फसेका छन्।'

In [32]:
from deep_translator import GoogleTranslator
from num2words import num2words

def translate_from_nepali_english(text, target_lang, reverse=False):
    if not reverse:
        # Split the sentence into words
        words = text.split()

        # Convert any numeric words to text using the num2words library
        for i, word in enumerate(words):
            if word.isnumeric():
                words[i] = num2words(int(word), lang='en')

        # Join the words back into a sentence and translate it using GoogleTranslator
        modified_text = ' '.join(words)
        translated_text = GoogleTranslator(source='auto', target=target_lang).translate(modified_text)

        return translated_text
    else:
        # Translate the text back to the original language
        translated_text = GoogleTranslator(source=target_lang, target='en').translate(text)

        # Replace any numeric text with their corresponding numerical values
        words = translated_text.split()
        for i, word in enumerate(words):
            try:
                numeric_value = int(num2words(word, lang='en'))
                words[i] = str(numeric_value)
            except ValueError:
                pass
        # Join the words back into a sentence and return it
        modified_text = ' '.join(words)
        return modified_text

# translate_from_nepali_english('अमेरिकी जनगणना ब्यूरोका अनुसार, टेक्सासमा 71.2 प्रतिशत हिस्पैनिक दर्ता मतदाता र 86 प्रतिशत भन्दा बढी अफ्रिकी अमेरिकी दर्ता मतदाताहरूले 2012 को चुनावमा भाग लिए।' , 'en')


In [68]:
new_df1['english_translation'] = new_df1['nepali_translation'].apply(lambda x: translate_from_nepali_english(x,"en"))


In [ ]:
new_df.head()

In [ ]:
new_df.to_csv('/kaggle/working/translated_data_english.tsv', sep='\t', index=False)


In [ ]:
# import string
# def removepunctuation(sentence):
#    new_text = sentence.replace("—", " ") # "—" in the text was not equivalent to "-" so we replcaed "—" with a space
#    print 
#   #iterate over every single character to see if it is a punctuation or not, and then concatenate them using "join"
#    punctuationfree="".join([i for i in new_text if i not in string.punctuation])
#    return punctuationfree

# removepunctuation('We, the bishops of the United States—can you believe it—came out in nineteen nineteen for more affordable, more comprehensive, more universal health care.')
